# Trabajo de Inteligencia artificial
 ## Análisis de noticias

 Realizado por:
 - Marta Aguilar Morcillo
 - Candela Jazmín Gutiérrez González

Fecha: 30/05/2025

Convocatoria de junio.

 ## 1. Lectura de datos

 Se comenzará con la lectura del corpus.

In [29]:
!pip install nltk
import nltk

from nltk import download

download('punkt_tab')                           # Tokenización
nltk.download('averaged_perceptron_tagger')     # POS tagging
nltk.download('averaged_perceptron_tagger_eng') # POS tagging
nltk.download('wordnet')                        # WordNet lemmatizer
nltk.download('omw-1.4')                        # WordNet multilingüe

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [31]:
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.data import path
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

path.append(".")

In [32]:
!pip install contractions
import contractions

In [33]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
import re
from bs4 import MarkupResemblesLocatorWarning
import warnings

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier

In [35]:
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

In [36]:
palabras_vacias_ingles = stopwords.words('english')

In [37]:
def elimina_html(contenido):
    return BeautifulSoup(contenido).get_text()

def elimina_no_alfanumerico(contenido):
    return [re.sub(r'[^\w]', '', palabra)
            for palabra in contenido
            if re.search(r'\w', palabra)]

def expandir_constracciones(contenido):
    return contractions.fix(contenido)

def pasar_a_minuscula(contenido):
    return contenido.lower()

def limpiar_texto(texto):
    texto = re.sub(r'[^a-zA-Z\s]', ' ', texto)  # Reemplaza todo lo que no es letra o espacio con espacio
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def elimina_palabras_vacias(contenido):
    return [palabra for palabra in contenido if palabra not in palabras_vacias_ingles]

def lematizador(contenido):
    lemmatizer = WordNetLemmatizer()
    pos_tags = pos_tag(contenido)

    resultado = []
    for palabra, tag in pos_tags:
        if tag.startswith('VB'):  # Verbos
            resultado.append(lemmatizer.lemmatize(palabra, pos='v'))  # infinitivo
        else:  # Sustantivos y el resto tal como están
            resultado.append(palabra)

    return resultado

In [38]:
def proceso_contenido(texto):
    texto = elimina_html(texto)
    texto = expandir_constracciones(texto)
    texto = pasar_a_minuscula(texto)
    texto = limpiar_texto(texto)                # Limpiar antes de tokenizar
    tokens = word_tokenize(texto)               
    tokens = elimina_no_alfanumerico(tokens)    # Limpiar tokens individuales
    tokens = elimina_palabras_vacias(tokens)
    tokens = lematizador(tokens)
    return tokens

In [39]:
# Cargar CSV con codificación segura y punto y coma como separador
df = pd.read_csv("news_corpus.csv", encoding="latin-1", sep=";", quotechar='"')

# Verificar que tenga exactamente 3 columnas
assert df.shape[1] == 3, "El CSV no tiene exactamente 3 columnas. Revisa el separador o las comillas."

resultados = []

for index, fila in df.iterrows():
    autor = fila.iloc[0]
    titulo = fila.iloc[1]
    cuerpo = fila.iloc[2]

    autor_proc = proceso_contenido(autor)
    titulo_proc = proceso_contenido(titulo)
    cuerpo_proc = proceso_contenido(cuerpo)

    # Unir las tres listas en una sola lista combinada
    fila_combinada = autor_proc + titulo_proc + cuerpo_proc
    resultados.append(fila_combinada)

In [40]:
# Mostrar los primeros 5 documentos procesados
for i, documento in enumerate(resultados[:5]):
    print(f"Documento {i+1}:")
    print(" - Palabras:", documento)
    print()

Documento 1:
 - Palabras: ['chhavi', 'tyagi', 'daman', 'diu', 'revoke', 'mandatory', 'rakshabandhan', 'offices', 'order', 'daman', 'diu', 'administration', 'wednesday', 'withdraw', 'circular', 'ask', 'women', 'staff', 'tie', 'rakhis', 'male', 'colleagues', 'order', 'trigger', 'backlash', 'employees', 'rip', 'apart', 'social', 'media', 'union', 'territory', 'administration', 'force', 'retreat', 'within', 'hours', 'issue', 'circular', 'make', 'compulsory', 'staff', 'celebrate', 'rakshabandhan', 'workplace', 'decide', 'celebrate', 'festival', 'rakshabandhan', 'august', 'connection', 'offices', 'departments', 'shall', 'remain', 'open', 'celebrate', 'festival', 'collectively', 'suitable', 'time', 'wherein', 'lady', 'staff', 'shall', 'tie', 'rakhis', 'colleagues', 'order', 'issue', 'august', 'gurpreet', 'singh', 'deputy', 'secretary', 'personnel', 'say', 'ensure', 'one', 'skipped', 'office', 'attendance', 'report', 'send', 'government', 'next', 'even', 'two', 'notifications', 'one', 'mandate